In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/extract1.gtf', sep='\t', skiprows=5, 
                 names=["seqname", "source", "feature", "start", "end", "score", "strand", 
                        "frame", "attributes"])
df.head()

,seqname,source,feature,start,end,score,strand,frame,attributes
0,3,havana,gene,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; g..."
1,3,havana,transcript,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
2,3,havana,exon,58572744,58572790,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
3,3,havana,exon,58573477,58573650,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
4,3,havana,exon,58574191,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."


Il campo attributes contiene in realtà una serie di altri attributi, divisi da un punto e virgola ```;```.
In questo caso specifico dovrebbero essere gli stessi attributi per ogni riga, ma è più sicuro scrivere un programma che calcola tutti i campi: ```nuovi_campi```

In [3]:
nuovi_campi = set()
for a in df['attributes']:
    for field in re.split("; ", a):
        m = re.search("^\S+", field)
        if m:
            nuovi_campi.add(m.group(0))
nuovi_campi

{'ccds_id',
 'exon_id',
 'exon_number',
 'exon_version',
 'gene_biotype',
 'gene_id',
 'gene_name',
 'gene_source',
 'gene_version',
 'havana_gene',
 'havana_gene_version',
 'havana_transcript',
 'havana_transcript_version',
 'protein_id',
 'protein_version',
 'tag',
 'transcript_biotype',
 'transcript_id',
 'transcript_name',
 'transcript_source',
 'transcript_support_level',
 'transcript_version'}

Scriviamo adesso una funzione che riceve il campo ```attributes``` e uno dei campi aggiuntivi e ne restituisce il valore.

In [4]:
def estrai_nuovo_campo(attributi, nuovo_campo):
    m = re.search(nuovo_campo + ' \"(.*?)\"', attributi)
    if m:
        return m.group(1)
    else:
        return None
estrai_nuovo_campo('gene_id "ENSG00000243384"; gene_version "1";', "gene_version")

'1'

Adesso posso creare i nuovi campi nel dataframe. La funzione da applicare è ```apply``` che viene applicata ad una colonna e riceve come argomento una funzione.

```lambda``` è un modo di creare una nuova funzione senza dargli un nome esplicito, come invece viene fatto con la standard ```def```

In [5]:
for campo in nuovi_campi:
    df[campo] = df["attributes"].apply(lambda x: estrai_nuovo_campo(x, campo))

In [6]:
df.head()

,seqname,source,feature,start,end,score,strand,frame,attributes,transcript_version,...,havana_gene,gene_source,gene_name,exon_version,tag,gene_id,transcript_name,transcript_biotype,havana_transcript_version,ccds_id
0,3,havana,gene,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; g...",None,...,OTTHUMG00000159155,havana,RP11-475O23.2,None,None,ENSG00000243384,None,None,None,None
1,3,havana,transcript,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",1,...,OTTHUMG00000159155,havana,RP11-475O23.2,None,basic,ENSG00000243384,RP11-475O23.2-001,antisense,1,None
2,3,havana,exon,58572744,58572790,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",1,...,OTTHUMG00000159155,havana,RP11-475O23.2,1,basic,ENSG00000243384,RP11-475O23.2-001,antisense,1,None
3,3,havana,exon,58573477,58573650,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",1,...,OTTHUMG00000159155,havana,RP11-475O23.2,1,basic,ENSG00000243384,RP11-475O23.2-001,antisense,1,None
4,3,havana,exon,58574191,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",1,...,OTTHUMG00000159155,havana,RP11-475O23.2,1,basic,ENSG00000243384,RP11-475O23.2-001,antisense,1,None


Estraggo solo le righe relative ad un gene.
Notare l'uso di ```copy``` per creare un nuovo dataframe su cui lavorare.

In [7]:
geni = df.query("feature == 'gene'").copy()
type(geni)

pandas.core.frame.DataFrame

In [8]:
geni.drop(['attributes', 'feature', 'exon_id', 'ccds_id'], axis=1, inplace=True)
geni.head()

,seqname,source,start,end,score,strand,frame,transcript_version,transcript_source,transcript_support_level,...,havana_transcript,havana_gene,gene_source,gene_name,exon_version,tag,gene_id,transcript_name,transcript_biotype,havana_transcript_version
0,3,havana,58572744,58574319,.,+,.,None,None,None,...,None,OTTHUMG00000159155,havana,RP11-475O23.2,None,None,ENSG00000243384,None,None,None
5,3,havana,58607080,58634440,.,+,.,None,None,None,...,None,OTTHUMG00000159160,havana,FAM3D-AS1,None,None,ENSG00000244383,None,None,None
11,3,ensembl_havana,58633946,58666848,.,-,.,None,None,None,...,None,OTTHUMG00000159148,ensembl_havana,FAM3D,None,None,ENSG00000198643,None,None,None
109,3,ensembl_havana,58717365,59050084,.,-,.,None,None,None,...,None,OTTHUMG00000159151,ensembl_havana,C3orf67,None,None,ENSG00000163689,None,None,None
356,3,havana,58824437,59019093,.,+,.,None,None,None,...,None,OTTHUMG00000159203,havana,C3orf67-AS1,None,None,ENSG00000242428,None,None,None


In [9]:
geni.index

Int64Index([    0,     5,    11,   109,   356,   373,   377,   382,   386,
              389,
            ...
            56126, 56131, 56143, 56446, 56449, 56452, 56456, 56459, 56479,
            56482],
           dtype='int64', length=1300)

In [10]:
for row_a in geni.itertuples():
    for row_b in geni.itertuples():
            if (row_a.gene_id != row_b.gene_id and (row_a.start <= row_b.start <= row_a.end or
                                                   row_b.start <= row_a.start <= row_b.end)):
                print(row_a.Index, row_a. gene_id, row_a.gene_name, row_a.start, row_a.end,
                     row_b.Index, row_b. gene_id, row_b.gene_name, row_b.start, row_b.end)

5 ENSG00000244383 FAM3D-AS1 58607080 58634440 11 ENSG00000198643 FAM3D 58633946 58666848
11 ENSG00000198643 FAM3D 58633946 58666848 5 ENSG00000244383 FAM3D-AS1 58607080 58634440
109 ENSG00000163689 C3orf67 58717365 59050084 356 ENSG00000242428 C3orf67-AS1 58824437 59019093
356 ENSG00000242428 C3orf67-AS1 58824437 59019093 109 ENSG00000163689 C3orf67 58717365 59050084
392 ENSG00000189283 FHIT 59749310 61251459 506 ENSG00000240097 RP11-641C17.4 60616822 60618079
392 ENSG00000189283 FHIT 59749310 61251459 509 ENSG00000281426 MIR548BB 60617805 60617870
392 ENSG00000189283 FHIT 59749310 61251459 512 ENSG00000232407 RP11-641C17.2 60690225 60690719
392 ENSG00000189283 FHIT 59749310 61251459 515 ENSG00000225673 RP11-641C17.3 60730055 60730644
392 ENSG00000189283 FHIT 59749310 61251459 518 ENSG00000244183 RP11-641C17.1 60732144 60732636
392 ENSG00000189283 FHIT 59749310 61251459 521 ENSG00000212211 U3 60856389 60856605
506 ENSG00000240097 RP11-641C17.4 60616822 60618079 392 ENSG00000189283 FHIT

In [17]:
geni.loc[[512, 515, 518]]

,seqname,source,start,end,score,strand,frame,transcript_version,transcript_source,transcript_support_level,...,havana_transcript,havana_gene,gene_source,gene_name,exon_version,tag,gene_id,transcript_name,transcript_biotype,havana_transcript_version
512,3,havana,60690225,60690719,.,-,.,None,None,None,...,None,OTTHUMG00000158654,havana,RP11-641C17.2,None,None,ENSG00000232407,None,None,None
515,3,havana,60730055,60730644,.,-,.,None,None,None,...,None,OTTHUMG00000158655,havana,RP11-641C17.3,None,None,ENSG00000225673,None,None,None
518,3,havana,60732144,60732636,.,-,.,None,None,None,...,None,OTTHUMG00000158653,havana,RP11-641C17.1,None,None,ENSG00000244183,None,None,None


In [30]:
def s1():
    lista = []
    for row_a in geni.itertuples():
            for row_b in geni.itertuples():
                if (row_a.gene_id != row_b.gene_id and row_a.start <= row_b.start <= row_a.end):
                    lista.append({'a': row_a.gene_id, 'b': row_b.gene_id})
%timeit s1()

1 loop, best of 3: 5.5 s per loop


Invece di fare due cicli nidificati, uso una query per estrarre solo le righe che interessano.

In [ ]:
for row_a in geni.itertuples():
            df_overlap = geni.query("@row_a.gene_id != gene_id and @row_a.start <= start <= @row_a.end")

In [50]:
def s2():
    lista = []
    for row_a in geni.itertuples():
            df_overlap = geni.query("@row_a.gene_id != gene_id and @row_a.start <= start <= @row_a.end")
            [ lista.append({'a': row_a.gene_id, 'b': b}) for b in df_overlap['gene_id'] ]
    return lista

print(s2())

[{'a': 'ENSG00000244383', 'b': 'ENSG00000198643'}, {'a': 'ENSG00000163689', 'b': 'ENSG00000242428'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000240097'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000281426'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000232407'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000225673'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000244183'}, {'a': 'ENSG00000189283', 'b': 'ENSG00000212211'}, {'a': 'ENSG00000240097', 'b': 'ENSG00000281426'}, {'a': 'ENSG00000144724', 'b': 'ENSG00000252420'}, {'a': 'ENSG00000144724', 'b': 'ENSG00000226360'}, {'a': 'ENSG00000144724', 'b': 'ENSG00000252184'}, {'a': 'ENSG00000144724', 'b': 'ENSG00000237456'}, {'a': 'ENSG00000144724', 'b': 'ENSG00000241472'}, {'a': 'ENSG00000241472', 'b': 'ENSG00000114405'}, {'a': 'ENSG00000163618', 'b': 'ENSG00000225662'}, {'a': 'ENSG00000163618', 'b': 'ENSG00000244632'}, {'a': 'ENSG00000163618', 'b': 'ENSG00000252449'}, {'a': 'ENSG00000244342', 'b': 'ENSG00000242841'}, {'a': 'ENSG00000163630', 'b': 'ENSG00000241359'},

In [51]:
%timeit s2()

1 loop, best of 3: 2.57 s per loop


Per ogni gene, contare quanti trascritti contiene

In [57]:
df_transcript = df.query("feature == 'transcript'").copy()
conteggio = pd.DataFrame(df_transcript.groupby("gene_id").size(), columns=['conteggio'])

In [58]:
type(conteggio)

pandas.core.frame.DataFrame

In [59]:
conteggio.query("conteggio >= 3")

,conteggio
gene_id,
ENSG00000004399,17
ENSG00000014257,9
ENSG00000017260,24
ENSG00000018408,12
ENSG00000034533,8
ENSG00000036054,9
ENSG00000044524,3
ENSG00000047457,15
ENSG00000051341,4
